In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display

import duckdb
import pandas as pd
from pathlib import Path
import logging
from Bio import SeqIO
from datetime import datetime
import time

from typing import List, Optional, Union, Dict, Set

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [2]:
from planter.database.query_manager import DatabaseManager
db_path = "/mnt/data2/planter_outputs/planter2.duckdb"

with DatabaseManager(db_path) as db_manager:
    summary = db_manager.query_manager.database_summary()
    display(summary)

,total_sequences,total_samples,representative_sequences,annotated_sequences,sequences_with_go,sequences_with_ec,total_clusters,avg_sequence_length,min_sequence_length,max_sequence_length
0,2053686,79,0,1691662,809670,399851,0,429.87,100,7993


# Logan

https://github.com/IndexThePlanet/Logan/blob/main/Unitigs.md

In [12]:
all_paired_lichen_md = pd.read_csv('/home/ubuntu/lichen/data/All Paired Lichen Metadata.csv')
all_paired_lichen_md

/tmp/ipykernel_530070/172446834.py:1: DtypeWarning: Columns (55,56,61,67,68,75,84,88,89,90,91,92,93,94,96,99,101,103,104,105,106,107,108,111,112,113,114,115,116,117,118,122,123,124,125,133,134,135,136,137,139,140,141,143) have mixed types. Specify dtype option on import or set low_memory=False.
  all_paired_lichen_md = pd.read_csv('/home/ubuntu/lichen/data/All Paired Lichen Metadata.csv')


,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Center Name,Consent,DATASTORE filetype,...,Nutrient,O_Depth_cm,project_id,Severity_Class,Veg_Comm,Clay_pct,plant-associated_environmental_package,Sand_pct,Silt_pct,elevation
0,SRR18162807,WGS,300,18930750600,PRJNA795879,SAMN25961858,7890425392,UNIVERSITY OF GRAZ,public,"fastq,run.zq,sra",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SRR18162808,WGS,250,13332657500,PRJNA795879,SAMN25961857,6375778834,UNIVERSITY OF GRAZ,public,"fastq,run.zq,sra",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SRR18162809,WGS,298,9844940474,PRJNA795879,SAMN25961856,4477451125,UNIVERSITY OF GRAZ,public,"fastq,run.zq,sra",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SRR18162810,WGS,300,26592286200,PRJNA795879,SAMN25961855,11219417482,UNIVERSITY OF GRAZ,public,"fastq,run.zq,sra",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SRR18162811,WGS,300,25018821000,PRJNA795879,SAMN25961854,10130400280,UNIVERSITY OF GRAZ,public,"fastq,run.zq,sra",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5657,ERR4235148,RNA-Seq,300,6290998800,PRJEB38537,SAMEA6943442,1926544514,THE JOHN INNES CENTRE,public,"sra,fastq,run.zq",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5658,SRR30709219,WGS,298,7251974718,PRJNA1162296,SAMN43808046,3225944768,YUNNAN ACADEMY OF FORESTRY AND GRASSLAND,public,"run.zq,sra,fastq",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5659,SRR31740197,AMPLICON,577,21158486,PRJNA1198499,SAMN45844533,11613048,M.V. LOMONOSOV MOSCOW STATE UNIVERSITY,public,"sra,run.zq,fastq",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5660,SRR31740198,AMPLICON,575,20951919,PRJNA1198499,SAMN45844532,11255736,M.V. LOMONOSOV MOSCOW STATE UNIVERSITY,public,"fastq,run.zq,sra",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
all_paired_lichen_md.pipe(lambda df: df[df['Assay Type'] == 'RNA-Seq'])

,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Center Name,Consent,DATASTORE filetype,...,Nutrient,O_Depth_cm,project_id,Severity_Class,Veg_Comm,Clay_pct,plant-associated_environmental_package,Sand_pct,Silt_pct,elevation
33,SRR18070778,RNA-Seq,202,2221595394,PRJNA210248,SAMN26082894,1359657955,SUNCHON NATIONAL UNIVERSITY,public,"fastq,run.zq,sra",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,SRR18070779,RNA-Seq,202,3326612760,PRJNA210248,SAMN26082894,2033913208,SUNCHON NATIONAL UNIVERSITY,public,"fastq,run.zq,sra",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,SRR18070780,RNA-Seq,202,2146875190,PRJNA210248,SAMN26082894,1313510753,SUNCHON NATIONAL UNIVERSITY,public,"fastq,run.zq,sra",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,SRR18070781,RNA-Seq,252,1760551884,PRJNA210248,SAMN26082894,656060602,SUNCHON NATIONAL UNIVERSITY,public,"fastq,run.zq,sra",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,SRR18070782,RNA-Seq,252,4279994460,PRJNA210248,SAMN26082894,1629760699,SUNCHON NATIONAL UNIVERSITY,public,"fastq,run.zq,sra",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5653,ERR4235144,RNA-Seq,300,6993738000,PRJEB38537,SAMEA6943438,2119973425,THE JOHN INNES CENTRE,public,"fastq,sra,run.zq",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5654,ERR4235145,RNA-Seq,300,6679726800,PRJEB38537,SAMEA6943439,2134456817,THE JOHN INNES CENTRE,public,"run.zq,fastq,sra",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5655,ERR4235146,RNA-Seq,300,7407111900,PRJEB38537,SAMEA6943440,2263747230,THE JOHN INNES CENTRE,public,"sra,run.zq,fastq",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5656,ERR4235147,RNA-Seq,300,6895473600,PRJEB38537,SAMEA6943441,2112586666,THE JOHN INNES CENTRE,public,"run.zq,sra,fastq",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
rnaseq_sra = all_paired_lichen_md.pipe(lambda df: df[df['Assay Type'] == 'RNA-Seq'])['Run'].tolist()

sra_ids = all_paired_lichen_md['Run'].tolist()


In [24]:
with open('/home/ubuntu/lichen/data/all_lichen_sra_ids.txt', 'w') as f:
    for sra_id in sra_ids:
        _ = f.write(f"{sra_id}\n")
